In [6]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("June9Assignment2").getOrCreate()
spark

In [8]:
#Read all 3 files (CSV + JSON) using PySpark.
df_employees=spark.read.csv("employees.csv",header=True,inferSchema=True)
df_employees.show()
df_attendance=spark.read.csv("attendance.csv",header=True,inferSchema=True)
df_attendance.show()
df_bonuses=spark.read.json("bonuses.json")
df_bonuses.show()
#Show schemas and sample records.
df_employees.printSchema()
df_attendance.printSchema()
df_bonuses.printSchema()
#Count distinct departments.
df_employees.select("department").distinct().count()

+-----+------+-----------+----------+------+---------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|
+-----+------+-----------+----------+------+---------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|
|    2|   Raj|Engineering|2020-03-15| 80000|        1|
|    3|Simran|Engineering|2022-07-10| 75000|        1|
|    4| Aamir|  Marketing|2019-11-20| 60000|        1|
|    5| Nisha|         HR|2023-01-05| 50000|        1|
+-----+------+-----------+----------+------+---------+

+-----+----------+-------+
|EmpID|      Date| Status|
+-----+----------+-------+
|    1|2024-04-01|Present|
|    1|2024-04-02|Present|
|    2|2024-04-01| Absent|
|    2|2024-04-02|Present|
|    3|2024-04-01|Present|
|    3|2024-04-02|Present|
|    4|2024-04-01| Absent|
|    4|2024-04-02| Absent|
|    5|2024-04-01|Present|
|    5|2024-04-02|Present|
+-----+----------+-------+

+-----+-----+----+
|Bonus|EmpID|Year|
+-----+-----+----+
| 5000|    1|2023|
| 7000|    2|2023|
| 6500|    3|2023|
| 6000|    

3

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.functions import datediff, current_date, round

#Add a column TenureYears using datediff() and round().
df_employees = df_employees.withColumn("TenureYears", round(datediff(current_date(), df_employees.JoinDate) / 365, 1))
df_employees.show()
#Calculate TotalCompensation = Salary + Bonus.
df_employees_with_bonus = df_employees.join(df_bonuses, on="EmpID", how="left")
df_employees= df_employees_with_bonus.withColumn("TotalCompensation", df_employees_with_bonus.Salary + df_employees_with_bonus.Bonus)
df_employees.show()
#Filter employees with more than 2 years in the company.
df_employees.filter(df_employees_with_bonus.TenureYears > 2).show()
#Show employees who report to a manager (ManagerID is not null).
df_employees.filter(df_employees_with_bonus.ManagerID.isNotNull()).show()

+-----+------+-----------+----------+------+---------+-----------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|
+-----+------+-----------+----------+------+---------+-----------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|        4.1|
|    2|   Raj|Engineering|2020-03-15| 80000|        1|        5.2|
|    3|Simran|Engineering|2022-07-10| 75000|        1|        2.9|
|    4| Aamir|  Marketing|2019-11-20| 60000|        1|        5.6|
|    5| Nisha|         HR|2023-01-05| 50000|        1|        2.4|
+-----+------+-----------+----------+------+---------+-----------+

+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|TotalCompensation|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|        4.1| 5000|2023|            60000|
|    2|   Ra

In [12]:
#Average salary per department.
df_employees.groupBy("Department").agg(F.avg("Salary")).show()
#Number of employees under each manager.
df_employees.groupBy("ManagerID").count().show()
#Count of absences per employee.
df_attendance.groupBy("EmpID").agg(F.count(F.when(df_attendance.Status == "Absent", 1).otherwise(None)).alias("AbsentCount")).show()

+-----------+-----------+
| Department|avg(Salary)|
+-----------+-----------+
|Engineering|    77500.0|
|         HR|    52500.0|
|  Marketing|    60000.0|
+-----------+-----------+

+---------+-----+
|ManagerID|count|
+---------+-----+
|     NULL|    1|
|        1|    4|
+---------+-----+

+-----+-----------+
|EmpID|AbsentCount|
+-----+-----------+
|    1|          0|
|    3|          0|
|    5|          0|
|    4|          2|
|    2|          1|
+-----+-----------+



In [16]:
#Join employees and attendance → Get attendance % (Present days / Total days).
from pyspark.sql import functions as F
from pyspark.sql.functions import round
df_employees_with_attendance = df_employees.join(df_attendance, on="EmpID", how="left")
df_employees_with_attendance = df_employees_with_attendance.groupBy("EmpID").agg(F.count(F.when(df_attendance.Status == "Present", 1)).alias("PresentDays"),F.count(F.when(df_attendance.Status.isNotNull(), 1)).alias("TotalDays"))
df_employees_with_attendance = df_employees_with_attendance.withColumn("AttendancePercentage", round((F.col("PresentDays") / F.col("TotalDays")) * 100, 2))
df_employees_with_attendance.show()

#Join employees and bonuses → Show top 3 employees by TotalCompensation.
df_bonuses_renamed = df_bonuses.withColumnRenamed("Bonus", "Bonus_renamed")
df_employees_with_bonuses = df_employees.join(df_bonuses_renamed, on="EmpID", how="left")
df_employees_with_bonuses = df_employees_with_bonuses.withColumn("TotalCompensation", df_employees_with_bonuses.Salary + df_employees_with_bonuses["Bonus_renamed"])
df_employees_with_bonuses.orderBy(F.desc("TotalCompensation")).limit(3).show()
#Multi-level join: employees + bonuses + attendance.
df_employees_with_bonuses_and_attendance = df_employees_with_bonuses.join(df_employees_with_attendance, on="EmpID", how="left")
df_employees_with_bonuses_and_attendance.show()

+-----+-----------+---------+--------------------+
|EmpID|PresentDays|TotalDays|AttendancePercentage|
+-----+-----------+---------+--------------------+
|    1|          2|        2|               100.0|
|    3|          2|        2|               100.0|
|    5|          2|        2|               100.0|
|    4|          0|        2|                 0.0|
|    2|          1|        2|                50.0|
+-----+-----------+---------+--------------------+

+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+-------------+----+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|TotalCompensation|Bonus_renamed|Year|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+-------------+----+
|    2|   Raj|Engineering|2020-03-15| 80000|        1|        5.2| 7000|2023|            87000|         7000|2023|
|    3|Simran|Engineering|2022-07-10| 75000|        1|        2.9| 6500|2023|   

In [18]:
#Extract year and month from JoinDate.
from pyspark.sql.functions import year,month,col
df_employees = df_employees.withColumn("JoinYear", year(df_employees.JoinDate)).withColumn("JoinMonth", month(df_employees.JoinDate))
df_employees.show()
#Mask employee names using regex.
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

def mask_name(name):
    name_parts = name.split(' ')
    if len(name_parts) == 2:
        first_name, last_name = name_parts
        masked_first_name = first_name[0] + '*' * (len(first_name) - 1)
        masked_last_name = last_name[0] + '*' * (len(last_name) - 1)
        return f"{masked_first_name} {masked_last_name}"
    elif len(name_parts) == 1:
        return name[0] + '*' * (len(name) - 1)
    else:
        return name

mask_name_udf = udf(mask_name, StringType())
customers_df = df_employees.withColumn("Name", mask_name_udf(col("Name")))
customers_df.show()

#Use substring() to create EmpCode like "EMP001".
from pyspark.sql.functions import col, lpad,concat,lit
df_employees_with_code = df_employees.withColumn("EmpCode",lpad("EmpID", 3, "0"))
df_employees_with_code = df_employees_with_code.withColumn("EmpCode",concat(lit("EMP"), col("EmpCode")))
df_employees_with_code.show()

+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+--------+---------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|TotalCompensation|JoinYear|JoinMonth|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+--------+---------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|        4.1| 5000|2023|            60000|    2021|        5|
|    2|   Raj|Engineering|2020-03-15| 80000|        1|        5.2| 7000|2023|            87000|    2020|        3|
|    3|Simran|Engineering|2022-07-10| 75000|        1|        2.9| 6500|2023|            81500|    2022|        7|
|    4| Aamir|  Marketing|2019-11-20| 60000|        1|        5.6| 6000|2023|            66000|    2019|       11|
|    5| Nisha|         HR|2023-01-05| 50000|        1|        2.4| 4000|2023|            54000|    2023|        1|
+-----+------+-----------+----------+------+---------+-----------+-----+----+---

In [21]:
#Use when/otherwise to label performance:
#“High” if Bonus > 6000
#“Medium” if 4000–6000
#“Low” otherwise
from pyspark.sql.functions import when
df_employees_with_performance = df_employees.withColumn("Performance", when(df_employees.Bonus > 6000, "High").when((df_employees.Bonus >= 4000) & (df_employees.Bonus <= 6000), "Medium").otherwise("Low"))
df_employees_with_performance.show()
#Handle missing ManagerID using fillna("No Manager").
df_employees = df_employees.fillna({"ManagerID": -1})
df_employees.show()
df_employees = df_employees.fillna({"ManagerID": "No Manager"})
df_employees.show()

+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+--------+---------+-----------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|TotalCompensation|JoinYear|JoinMonth|Performance|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+--------+---------+-----------+
|    1| Anita|         HR|2021-05-01| 55000|       -1|        4.1| 5000|2023|            60000|    2021|        5|     Medium|
|    2|   Raj|Engineering|2020-03-15| 80000|        1|        5.2| 7000|2023|            87000|    2020|        3|       High|
|    3|Simran|Engineering|2022-07-10| 75000|        1|        2.9| 6500|2023|            81500|    2022|        7|       High|
|    4| Aamir|  Marketing|2019-11-20| 60000|        1|        5.6| 6000|2023|            66000|    2019|       11|     Medium|
|    5| Nisha|         HR|2023-01-05| 50000|        1|        2.4| 4000|2023|            54000|    2023|       

In [27]:
# create and use database hr.
spark.sql("create database if not exists hr")
spark.sql("use hr")
# save all dataframes as tables: employees, attendance, bonuses.
df_employees.write.mode('overwrite').saveAsTable("employees")
df_attendance.write.mode('overwrite').saveAsTable("attendance")
df_bonuses.write.mode('overwrite').saveAsTable("bonuses")
# write sql queries:
# top paid employee in each department.
spark.sql("""select Department, max(Salary) as max_salary from employees group by Department""").show()
# attendance rate by department.
spark.sql("""select Department, count(case when status = 'Present' then 1 end) / count(*) as attendance_rate from employees e join attendance a on e.EmpID = a.EmpID group by Department""").show()
# employees joined after 2021 with salary > 70,000.
spark.sql("""select * from employees where JoinDate > '2021-01-01' and Salary > 70000""").show()

+-----------+----------+
| Department|max_salary|
+-----------+----------+
|Engineering|     80000|
|         HR|     55000|
|  Marketing|     60000|
+-----------+----------+

+-----------+---------------+
| Department|attendance_rate|
+-----------+---------------+
|Engineering|           0.75|
|         HR|            1.0|
|  Marketing|            0.0|
+-----------+---------------+

+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+--------+---------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|TotalCompensation|JoinYear|JoinMonth|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+--------+---------+
|    3|Simran|Engineering|2022-07-10| 75000|        1|        2.9| 6500|2023|            81500|    2022|        7|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+--------+---------+



In [28]:
#Use a UDF to classify department as "Tech" vs "Non-Tech".
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def classify_department(department):
    if department in ["Engineering"]:
        return "Tech"
    else:
        return "Non-Tech"

classify_department_udf = udf(classify_department, StringType())
df_employees = df_employees.withColumn("DepartmentType", classify_department_udf(df_employees.Department))
df_employees.show()
#Create a view emp_attendance_summary.
df_employees.createOrReplaceTempView("emp_attendance_summary")
#Save it as Parquet partitioned by Department.
df_employees.write.mode('overwrite').partitionBy("Department").parquet("emp_attendance_summary")

+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+--------+---------+--------------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|TotalCompensation|JoinYear|JoinMonth|DepartmentType|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+--------+---------+--------------+
|    1| Anita|         HR|2021-05-01| 55000|       -1|        4.1| 5000|2023|            60000|    2021|        5|      Non-Tech|
|    2|   Raj|Engineering|2020-03-15| 80000|        1|        5.2| 7000|2023|            87000|    2020|        3|          Tech|
|    3|Simran|Engineering|2022-07-10| 75000|        1|        2.9| 6500|2023|            81500|    2022|        7|          Tech|
|    4| Aamir|  Marketing|2019-11-20| 60000|        1|        5.6| 6000|2023|            66000|    2019|       11|      Non-Tech|
|    5| Nisha|         HR|2023-01-05| 50000|        1|        2.4| 4000|2023|            5